# *btest* initial steps

step-by-step

In [1]:
import argparse
import csv
import itertools
import logging
import os
import sys
import shutil
import time
import math
import random
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import math
from scipy.stats import spearmanr
from btest import config
from btest import utils
from btest import blockplot
from scipy.stats import t

In [2]:
method = 'spearman'
min_var = 0.0
fdr =0.1

In [3]:
dataX , dataY, featuresX, featuresY  = utils.readData('../demo/X_dataset.txt', '../demo/Y_dataset.txt', min_var=min_var)

In [39]:
%%time
within_X = utils.btest_corr(dataX, featuresX, method=method, fdr=fdr, Type='withinX')
within_Y  = utils.btest_corr(dataY, featuresY, method=method, fdr=fdr, Type='withinY')
dataAll = np.concatenate((dataX, dataY), axis=0)
X_Y = utils.btest_corr(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

CPU times: user 10.2 s, sys: 35.5 ms, total: 10.3 s
Wall time: 10.4 s


In [5]:
%%time
from btest import utils
from btest.utils import bh
within_X = utils.btest_corr_2(dataX, featuresX, method=method, fdr=fdr, Type='withinX')
within_Y  = utils.btest_corr_2(dataY, featuresY, method=method, fdr=fdr, Type='withinY')
dataAll = np.concatenate((dataX, dataY), axis=0)
X_Y = utils.btest_corr_2(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

CPU times: user 2 s, sys: 14.6 ms, total: 2.01 s
Wall time: 2.07 s


In [76]:
dataAll = dataX
features = featuresX
features_y = None
Type = 'withinX'
iRow = list(range(0, len(features)))
if Type == 'X_Y':
    iCol = list(range(len(features), len(features)+len(features_y)-1))
    features_y = features + features_y
else:
    features_y = features
    iCol = list(range(len(features)))
tests = []

# creating the complete dataset
dataAll2 = pd.DataFrame(dataAll.T, columns=features_y)
cr = dataAll2.corr(method=method)

# calculating t-statistics, based on the correlations
t_stat = (cr*(dataAll2.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll2.shape[0]-2))
#rho_long_format = cr.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'Correlation'})
#pval_long_format = pv.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'pval'})
#results = pd.concat([pval_long_format, rho_long_format["Correlation"]], axis=1)


In [78]:
cr = cr.to_numpy()
#pv = pv.to_numpy()

In [79]:
for i in iRow:
    for j in iCol:
        if i<=j:
            #print(i, j)
            #print(dataAll[i])
            X = dataAll[i]
            Y = dataAll[j]
            nas = np.logical_or(X != X, Y != Y)
            not_na = sum(~nas)
            #X = Y[~nas]
            #Y = Y[~nas]
            #new_X, new_Y = remove_pairs_with_a_missing(X, Y)
            correlation = cr[i, j]
            pval =  pv[i, j]
            tests.append([features[i], features_y[j], pval, correlation, not_na])
results = pd.DataFrame(tests, columns=['Feature_1','Feature_2','pval', 'Correlation', 'Not_NAs'])

p_adust, p_threshold = bh(results["pval"].values, fdr)

results["P_adusted"] = p_adust
results["bh_fdr_threshold"] = p_threshold
results['Type'] = Type
results = results.sort_values(['pval', 'Correlation'],
                              ascending=[True, False])

In [14]:
# feature names for the complete dataset
featuresAll = featuresX.copy()
featuresAll.extend(featuresY)

In [61]:
results

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
0,X0,X0,0.000000,1.000000,50,0.000000,1.110223e-15,withinX
100,X1,X1,0.000000,1.000000,49,0.000000,1.110223e-15,withinX
199,X2,X2,0.000000,1.000000,50,0.000000,1.110223e-15,withinX
297,X3,X3,0.000000,1.000000,50,0.000000,1.110223e-15,withinX
394,X4,X4,0.000000,1.000000,49,0.000000,1.110223e-15,withinX
...,...,...,...,...,...,...,...,...
3137,X38,X78,0.998917,-0.000197,50,0.999709,1.110223e-15,withinX
3149,X38,X90,0.998917,-0.000197,50,0.999511,1.110223e-15,withinX
3206,X39,X86,0.998917,-0.000197,50,0.999313,1.110223e-15,withinX
4435,X65,X80,0.998917,-0.000197,50,0.999115,1.110223e-15,withinX


In [15]:
# creating the complete dataset
dataAll_df = pd.DataFrame(dataAll.T, columns=featuresAll)

In [16]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

CPU times: user 13.1 ms, sys: 2.85 ms, total: 16 ms
Wall time: 17.6 ms


In [17]:
cr

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,Y90,Y91,Y92,Y93,Y94,Y95,Y96,Y97,Y98,Y99
X0,1.000000,0.904965,0.908268,0.926614,0.930308,0.932730,0.896831,0.926614,0.957192,0.908268,...,0.000972,-0.082436,-0.076321,-0.077985,-0.125818,-0.065445,-0.070205,-0.082436,-0.076321,-0.103537
X1,0.904965,1.000000,0.940657,0.936643,0.911301,0.904965,0.910148,0.936643,0.930308,0.930308,...,0.129303,0.018314,0.043772,0.047183,-0.015886,0.071999,-0.000779,0.037408,0.037408,0.003003
X2,0.908268,0.940657,1.000000,0.926614,0.928135,0.914384,0.921874,0.938845,0.926614,0.932730,...,0.073855,-0.009051,0.003180,0.013119,-0.059407,0.025115,-0.039628,-0.009051,-0.002935,-0.042054
X3,0.926614,0.936643,0.926614,1.000000,0.936643,0.920499,0.921874,0.963307,0.944961,0.932730,...,0.061708,-0.021282,-0.009051,-0.005102,-0.059407,0.013041,-0.033513,-0.027397,-0.015166,-0.048202
X4,0.930308,0.911301,0.928135,0.936643,1.000000,0.923972,0.948656,0.930308,0.911301,0.949315,...,-0.028619,-0.102612,-0.115341,-0.117056,-0.172823,-0.084938,-0.108976,-0.128070,-0.108976,-0.118559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y95,-0.065445,0.071999,0.025115,0.013041,-0.084938,-0.029221,-0.023085,-0.017146,-0.005071,-0.041295,...,0.934062,0.924677,0.924677,0.952050,0.904637,1.000000,0.912602,0.924677,0.918639,0.921260
Y96,-0.070205,-0.000779,-0.039628,-0.033513,-0.108976,-0.045744,-0.064431,-0.064090,-0.039628,-0.082436,...,0.918090,0.951076,0.951076,0.942385,0.924677,0.912602,1.000000,0.938845,0.938845,0.941667
Y97,-0.082436,0.037408,-0.009051,-0.027397,-0.128070,-0.064090,-0.051977,-0.045744,-0.021282,-0.057975,...,0.924164,0.963307,0.957192,0.966679,0.942788,0.924677,0.938845,1.000000,0.938845,0.935519
Y98,-0.076321,0.037408,-0.002935,-0.015166,-0.108976,-0.051859,-0.058204,-0.033513,-0.015166,-0.057975,...,0.912017,0.944961,0.963307,0.954532,0.936751,0.918639,0.938845,0.938845,1.000000,0.941667


In [27]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

CPU times: user 19.3 ms, sys: 2.56 ms, total: 21.9 ms
Wall time: 23.1 ms


In [10]:
X_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
165,X1,Y66,0.021416,0.327972,49,212.013466,0.65493,X_Y
174,X1,Y75,0.033062,0.305052,49,163.658486,0.65493,X_Y
2747,X27,Y74,0.036443,0.296654,50,120.262541,0.65493,X_Y
173,X1,Y74,0.040235,0.294122,49,99.582635,0.65493,X_Y
1526,X15,Y41,0.042550,0.287992,50,84.249800,0.65493,X_Y
...,...,...,...,...,...,...,...,...
7670,X77,Y47,0.999459,-0.000098,50,0.999863,0.65493,X_Y
2271,X22,Y93,0.999729,0.000049,50,1.000032,0.65493,X_Y
5255,X53,Y8,0.999729,0.000049,50,0.999931,0.65493,X_Y
8647,X87,Y34,0.999729,0.000049,50,0.999830,0.65493,X_Y


In [11]:
within_X

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
100,X1,X1,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
394,X4,X4,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
585,X6,X6,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
1134,X12,X12,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
1222,X13,X13,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
...,...,...,...,...,...,...,...,...
3137,X38,X78,0.998917,-0.000197,50,0.999709,9.957214e-16,withinX
3149,X38,X90,0.998917,-0.000197,50,0.999511,9.957214e-16,withinX
3206,X39,X86,0.998917,-0.000197,50,0.999313,9.957214e-16,withinX
4435,X65,X80,0.998917,-0.000197,50,0.999115,9.957214e-16,withinX


In [12]:
within_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
0,Y0,Y0,0.000000,1.000000,50,0.000000,0.035926,withinY
100,Y1,Y1,0.000000,1.000000,50,0.000000,0.035926,withinY
199,Y2,Y2,0.000000,1.000000,50,0.000000,0.035926,withinY
394,Y4,Y4,0.000000,1.000000,50,0.000000,0.035926,withinY
585,Y6,Y6,0.000000,1.000000,50,0.000000,0.035926,withinY
...,...,...,...,...,...,...,...,...
4002,Y54,Y87,0.993502,-0.001182,50,0.994290,0.035926,withinY
2553,Y29,Y88,0.994587,-0.000984,50,0.995179,0.035926,withinY
487,Y4,Y97,0.995128,-0.000886,50,0.995522,0.035926,withinY
2836,Y33,Y97,0.995128,-0.000886,50,0.995325,0.035926,withinY


In [9]:
X_Y.pval

165     0.021416
174     0.033062
2747    0.036443
173     0.040235
1526    0.042550
          ...   
7670    0.999459
2271    0.999729
5255    0.999729
8647    0.999729
9011    0.999729
Name: pval, Length: 9900, dtype: float64

In [41]:
outputpath = "/Users/rah/Documents/omicsEye/btest/demo/btest_output"
os.makedirs(outputpath, exist_ok=True)
X_Y.to_csv(outputpath + '/X_Y.tsv', sep="\t")
#rho_X = rho.iloc[0:500, 0:500]
#rho_X.to_csv(outputpath + '/simtable_X.tsv', sep="\t")
#rho_X_Y.to_csv(outputpath + '/simtable.tsv', sep="\t")
within_X.to_csv(outputpath + '/within_X.tsv', sep="\t")
within_Y.to_csv(outputpath + '/within_Y.tsv', sep="\t")

In [2]:
#associations = blockplot.load_associations(path = outputpath + '/X_Y.tsv')
associations = blockplot.load_associations('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new/X_Y.tsv')

In [3]:
outputpath

NameError: name 'outputpath' is not defined

In [4]:
associations

[['616', ['GCF00020'], ['cystine'], '0.0', '1.0', '0.0'],
 ['623', ['GCF00020'], ['dihydroorotate'], '0.0', '1.0', '0.0'],
 ['673',
  ['GCF00020'],
  ['hydrochlorothiazide_C18-neg_v2'],
  '0.0',
  '1.0',
  '0.0'],
 ['3031', ['GCF00023'], ['C34:2 DAG'], '0.0', '1.0', '0.0'],
 ['3091', ['GCF00023'], ['C52:7 TAG'], '0.0', '1.0', '0.0'],
 ['3327', ['GCF00023'], ['tartarate'], '0.0', '1.0', '0.0'],
 ['5361', ['GCF00099'], ['2-aminoheptanoic acid'], '0.0', '1.0', '0.0'],
 ['5366', ['GCF00099'], ['2-hydroxyglutarate'], '0.0', '1.0', '0.0'],
 ['5388', ['GCF00099'], ['4-pyridoxate'], '0.0', '1.0', '0.0'],
 ['5414', ['GCF00099'], ['alanylalanine'], '0.0', '1.0', '0.0'],
 ['5420', ['GCF00099'], ['alpha-ketoglutarate'], '0.0', '1.0', '0.0'],
 ['5429', ['GCF00099'], ['arginine_HILIC-pos_v3'], '0.0', '1.0', '0.0'],
 ['5450', ['GCF00099'], ['C14 carnitine'], '0.0', '1.0', '0.0'],
 ['5455', ['GCF00099'], ['C14:1 carnitine'], '0.0', '1.0', '0.0'],
 ['5461', ['GCF00099'], ['C16:0 LPC_C8-pos_v2'], '0.0',

In [5]:
#simtable = blockplot.load_order_table(outputpath + '/simtable.tsv', associations)
simtable = blockplot.load_order_table('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new//simtable.tsv', associations)

In [46]:
X_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold
365,X1,Y66,0.021416,0.327972,49,214.155016,0.65493
374,X1,Y75,0.033062,0.305052,49,165.311602,0.65493
5196,X27,Y74,0.036443,0.296654,50,121.477314,0.65493
373,X1,Y74,0.040235,0.294122,49,100.588520,0.65493
3021,X15,Y41,0.042550,0.287992,50,85.100808,0.65493
...,...,...,...,...,...,...,...
12544,X77,Y47,0.999459,-0.000098,50,0.999859,0.65493
4340,X22,Y93,0.999729,0.000049,50,1.000029,0.65493
9277,X53,Y8,0.999729,0.000049,50,0.999929,0.65493
13706,X87,Y34,0.999729,0.000049,50,0.999829,0.65493


In [47]:
blockplot.plot(
    simtable,
    associations,
    cmap="RdBu_r",
    mask=False,
    axlabels=["",""],
    outfile="/Users/rah/Documents/omicsEye/btest/demo/btest_output/blockplot.pdf",
    similarity="Spearman"
)